In [34]:
import random
import numpy as np
import keras.utils

We generate a sequence in the following data format:
* *x* and *y* describing a position in a grid of `100 x 100`
* *c* describing a control status with 3 possible states (0 = starting, 1 = holding, 2 = pausing)

State transitions follow this diagram:

```
+-+     +-+     +-+
|0| --> |1| --> |2|
+++  ^  +++  ^  +++
 ^   |   |   |   |
 |   |   |   |   |
 +---+---+   +---+
 |               |
 |               |
 +---------------+
```

This results in `100 x 100 x 3 = 30.000` possible one-hot encoded values ranging from 1 - 30.000.

In [146]:
STATES_COUNT = 3

STATE_STARTING = 0
STATE_HOLDING = 1
STATE_PAUSING = 2

DEFAULT_POSITION = [0, 0]


def random_position(grid_size):
    return [random.randint(0, grid_size - 1) for _ in range(2)]


def flip_position(pos):
    return [pos[1], pos[0]]
    

def next_state(previous_state):
    if previous_state == STATE_STARTING:
        next_state = STATE_HOLDING
    elif previous_state == STATE_HOLDING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_HOLDING,
            STATE_PAUSING
        ])
    elif previous_state == STATE_PAUSING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    else:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    return next_state
    

def generate_sequence(grid_size, seq_len):
    sequence = []
    current_state = None
    current_position = DEFAULT_POSITION
    for _ in range(seq_len):
        current_state = next_state(current_state)
        if current_state == STATE_STARTING:
            current_position = random_position(grid_size)
        elif current_state == STATE_PAUSING:
            current_position = DEFAULT_POSITION
        feature_vector = np.concatenate([current_position, [current_state]])
        sequence.append(feature_vector)
    return sequence


def generate_alternative_sequence(seq, grid_size):
    sequence = generate_sequence(grid_size, len(seq))
    for i in range(len(seq)):
        if sequence[i][2] == STATE_STARTING:
            # "react" to other sequence by flipping it
            flipped = flip_position([seq[i][0], seq[i][1]])
            # overwrite position of new sequence
            sequence[i][0] = flipped[0]
            sequence[i][1] = flipped[1]
    return sequence
    

def encode_sequence(seq, grid_size):
    # encode 3-d vector in index values
    m = np.zeros((grid_size, grid_size, STATES_COUNT))
    indexed = [np.ravel_multi_index(vector, m.shape) + 1 for vector in seq]
    # all x/y positions in grid * state variants + 1 start symbol
    n = (grid_size * grid_size * STATES_COUNT) + 1
    hot_encoded = keras.utils.to_categorical(indexed, num_classes=n)
    return hot_encoded


def decode_sequence(seq, grid_size):
    m = np.zeros((grid_size, grid_size, STATES_COUNT))
    one_hot_decoded = [np.argmax(vector) for vector in seq]
    decoded = [np.unravel_index(val - 1, m.shape) for val in one_hot_decoded]
    return decoded


def generate_dataset(grid_size, seq_len, n_samples):
    src_data, target_pad_data, target_data = [], [], []
    for _ in range(n_samples):
        # generate source sequence
        src = generate_sequence(grid_size, seq_len)
        src_encoded = encode_sequence(src, grid_size)
        # generate target sequence
        target = generate_alternative_sequence(src, grid_size)
        target_encoded = encode_sequence(target, grid_size)
        # generated target input sequence, begin with start symbol 0
        target_pad = target[:-1]
        target_pad_encoded = [0] + encode_sequence(target_pad, grid_size)
        # add to dataset
        src_data.append(src_encoded)
        target_data.append(target_encoded)
        target_pad_data.append(target_pad_encoded)
    return src_data, target_pad_data, target_data

In [147]:
grid_size = 100
seq_len = 1
n_samples = 1

# generate a simulated dataset
x1, x2, y = generate_dataset(grid_size, seq_len, n_samples)
print(decode_sequence(x1[0], grid_size))

[(0, 0, 2)]
